# SSPs Calibration: Kikstra and Rising

Our working local folders for this calibration are contained in the parent folder of this notebook, in _calibration/Kikstra-Rising_

In [3]:
using CSVFiles, Query, Downloads, DataFrames, XLSX, Interpolations

## Socioeconomic Data

We start by downloading the country level data from the SSP Database (Shared Socioeconomic Pathways) - Version 2.0 data to obtain _SspDb_country_data_2013-06-12_. Users will need to download these data to their local _calibration/Kikstra-Rising/data_ folder as the requires a (free) registration of a user account and email.  We get 5 SSPs from this source as follows.

- SSP1: IMAGE (PBL)
- SSP2: MESSAGE-GLOBIOM (IIASA)
- SSP3: AIM (NIES)
- SSP4: GCAM (PNNL)
- SSP5: REMIND-MAGPIE (PIK)

Source Citation: Keywan Riahi, Detlef P. van Vuuren, Elmar Kriegler, Jae Edmonds, Brian C. O’Neill, Shinichiro Fujimori, Nico Bauer, Katherine Calvin, Rob Dellink, Oliver Fricko, Wolfgang Lutz, Alexander Popp, Jesus Crespo Cuaresma, Samir KC, Marian Leimbach, Leiwen Jiang, Tom Kram, Shilpa Rao, Johannes Emmerling, Kristie Ebi, Tomoko Hasegawa, Petr Havlík, Florian Humpenöder, Lara Aleluia Da Silva, Steve Smith, Elke Stehfest, Valentina Bosetti, Jiyong Eom, David Gernaat, Toshihiko Masui, Joeri Rogelj, Jessica Strefler, Laurent Drouet, Volker Krey, Gunnar Luderer, Mathijs Harmsen, Kiyoshi Takahashi, Lavinia Baumstark, Jonathan C. Doelman, Mikiko Kainuma, Zbigniew Klimont, Giacomo Marangoni, Hermann Lotze-Campen, Michael Obersteiner, Andrzej Tabeau, Massimo Tavoni. 
The Shared Socioeconomic Pathways and their energy, land use, and greenhouse gas emissions implications: An overview, Global Environmental Change, Volume 42, Pages 153-168, 2017, ISSN 0959-3780, DOI:110.1016/j.gloenvcha.2016.05.009

Download URL: **https://tntcat.iiasa.ac.at/SspDb/dsd?Action=htmlpage&page=10**

These data are keyed by ISO 3166-1 alpha3 three-letter country codes, and we obtained a mapping key from the link below provided at the SSP URL to local folder _calibration/data/keys_.

Download URL: **https://tntcat.iiasa.ac.at/SspDb/static/download/ISO3166-1_codes_and_country_names.xlsx**

In [ ]:
download_url = 'https://tntcat.iiasa.ac.at/SspDb/static/download/ISO3166-1_codes_and_country_names.xlsx'
path = joinpath(@__DIR__, "data", "ISO3166-1_codes_and_country_names.xlsx")
Downloads.download(download_url, local_path)

The next step is to process these raw data using methodologies described [here](https://github.com/openmodels/MimiPAGE2020.jl/tree/master/preproc/growthrates) and [here](https://github.com/openmodels/SSP-Extensions) in R using the script _extend_ssp.R_ [TODO] add Scripts from Parthum and Tan to this repository for full replication.

In [ ]:
# TODO

Next, we postprocess the outputs of this file to produce subsetted CSV files, linearly interpolated to annual values, that we output to our home repository's _data`_ folder.

In [4]:
data = load(joinpath(@__DIR__, "data", "ssp_3000_projections.csv")) |> DataFrame
for model in unique(data[!, :model]), ssp in unique(data[!, :ssp])

    model_data = data |>
        @filter(_.model == model && _.ssp == ssp) |>
        DataFrame |>
        @select(:year, :region, :pop, :gdp) |>
        DataFrame

    model_data_interp = DataFrame(:year => [], :country => [], :pop => [], :gdp => [])
    all_years = collect(minimum(model_data.year):maximum(model_data.year))

    for region in unique(model_data.region)

        region_data = model_data |> @filter(_.region == region) |> @orderby(:year) |> DataFrame
        pop_itp = LinearInterpolation(region_data.year, region_data.pop)
        gdp_itp = LinearInterpolation(region_data.year, region_data.gdp)   
        
        append!(model_data_interp, DataFrame(
            :year => all_years,
            :country => fill(region, length(all_years)),
            :pop => pop_itp[all_years],
            :gdp => gdp_itp[all_years]
        ))
    end
    
    for (i, type) in enumerate([Int64, String, Float64, Float64])
        model_data_interp[:,i] = convert.(type, model_data_interp[:,i])
    end

    model_data_interp |> save(joinpath(@__DIR__, "..", "..", "data", "socioeconomic", "$(model)_$(ssp).csv"))
end

## Emissions Data

N/A